In [1]:
import cv2
import numpy as np

In [2]:
def pre_data():
  
    training = [11, 12, 13, 14, 15, 16, 17, 18]
    validation = [19, 20, 21, 23, 24, 25, 1, 4]
    test = [22, 2, 3, 5, 6, 7, 8, 9, 10]

    classes = ['walking','jogging','handwaving','boxing','running','handclapping']

    x_train = []
    y_train = []
    x_val, y_val = [], []
    x_test, y_test = [], []

    frames = 24

    sheet = open('00sequences.txt')

    for s in sheet:
        s = s.strip()
        if s == '' or s[0] != 'p':
            continue
        s = s.split()
        person = s[0][6:8]
        s[0] = s[0].split('_')
        typ = s[0][1]
        per = s[0][2][1]
        intyp = classes.index(typ)
        #     print(typ, per)
        for i in range(2, 6):
            try:
                start, end = s[i].strip(',').split('-')
                start, end = int(start), int(end)
                #       print(person, sftart, end)
                tmp = np.empty((frames, 120, 160, 3), dtype = np.uint8)
                for j in range(start, end+1):
                    if j - start >= frames:
                        break
                    im = cv2.imread("datalab/%s_img/d%s/person%s/frame%d.jpg"%(typ,per,person,j))
                    tmp[j - start] = im
                    #           print(im)
                    #         break
                    tmp = np.array(tmp)
                    #       print(tmp)
                if int(person) in training:
                    #         print(tmp.shape)
                    x_train.append(tmp)
                    y_train.append(intyp)
                elif int(person) in validation:
                    #         print(tmp.shape)
                    x_val.append(tmp)
                    y_val.append(intyp)
                elif int(person) in test:
                    #         print(tmp.shape)
                    x_test.append(tmp)
                    y_test.append(intyp)
            except:
                print(person, typ, intyp)
    x_train = np.asarray(x_train)
    y_train = np.asarray(y_train)
    x_val = np.asarray(x_val)
    y_val = np.asarray(y_val)
    x_test = np.asarray(x_test)
    y_test = np.asarray(y_test)
    #   print(x_train.shape, y_train)
    #   print(x_val.shape, y_val)
    return (x_train, y_train, x_val, y_val, x_test, y_test)
    #     break
  
x_train, y_train, x_val, y_val, x_test, y_test = pre_data()

('11', 'boxing', 3)
('13', 'handclapping', 5)
('13', 'handclapping', 5)
('13', 'handclapping', 5)
('13', 'handclapping', 5)
('13', 'handwaving', 2)
('14', 'handwaving', 2)
('15', 'boxing', 3)
('22', 'boxing', 3)


In [3]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [4]:
x_train = rgb2gray(x_train)
x_val = rgb2gray(x_val)
x_test = rgb2gray(x_test)

In [5]:
x_test.shape

(863, 24, 120, 160)

In [6]:
x_train = x_train.reshape(760,24,120,160,1)
x_val = x_val.reshape(768,24,120,160,1)
x_test = x_test.reshape(863,24,120,160,1)

y_train = y_train.reshape(760,1)
y_val = y_val.reshape(768,1)
y_test = y_test.reshape(863,1)

In [7]:
x_train.shape

(760, 24, 120, 160, 1)

In [8]:
from __future__ import print_function

from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import LSTM, TimeDistributed, Merge, Flatten
from keras.layers import Dense, Dropout, Activation, MaxPooling2D, MaxPooling3D
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional_recurrent import ConvLSTM2D
import numpy as np
# import pylab as plt

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
def create_model(frames):
    
    model = Sequential()
    model.add(BatchNormalization())
    model.add(ConvLSTM2D(filters=20,
                       kernel_size=(3, 3),             
                       input_shape = (frames, 120, 160, 1),  
                       return_sequences=True))
    
    model.add(Dropout(0.4))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(ConvLSTM2D(filters=20,
                       kernel_size=(3, 3),             
                       return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    model.add(ConvLSTM2D(filters=10,
                       kernel_size=(3, 3),             
                       return_sequences=False))
    
    model.add(Dropout(0.4))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(6,activation='softmax'))         
#     model.add(Dense(256,activation='relu'))
    
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [9]:
#model = create_model(24)
from keras.models import load_model
model = load_model('4model-001.h5')

In [11]:
y_train.shape

(760, 1)

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, 24, 118, 158, 20)  15200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 118, 158, 20)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 12, 59, 79, 20)    0         
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 12, 57, 77, 20)    28880     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 6, 28, 38, 20)     0         
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, 26, 36, 10)        10840     
_________________________________________________________________
dropout_2 (Dropout)          (None, 26, 36, 10)        0         
__________

In [13]:
import keras
checkpoint = keras.callbacks.ModelCheckpoint('7model-{epoch:03d}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
model.fit(x_train, y_train, batch_size = 20, epochs = 1, callbacks=[checkpoint], validation_data = (x_val, y_val))

Train on 760 samples, validate on 768 samples
Epoch 1/1
760/760 [==============================] - 2172s 3s/step - loss: 0.4558 - acc: 0.8276 - val_loss: 0.9435 - val_acc: 0.6276

Epoch 00001: val_loss improved from inf to 0.94355, saving model to 7model-001.h5


In [14]:
y_train.shape

(760, 1)

In [15]:
y_val.shape

(768, 1)

In [ ]:
#3model-001.h5 loss = 0.89181
#4model-001.h5 loss = 0.81537
#5model-001.h5 loss = 0.83995
#6model-001.h5 loss = 0.88543
#7model-001.h5 loss = 0.94355

In [10]:
scores = model.evaluate(x_test, y_test, verbose=1)

863/863 [==============================] - 374s 434ms/step


In [24]:
print("Accuracy = %.2f" % (scores[1]*100)+"%")

Accuracy = 64.66%
